<a href="https://colab.research.google.com/github/tatsuhiko-suyama/Something-/blob/main/5_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import numpy as np
import analysis_flow as af
import logging                       # ← これを追加
from scipy.optimize import minimize, Bounds, LinearConstraint
logging.basicConfig(level=logging.INFO,
                    format="%(levelname)s: %(message)s")

# ---- 0. グローバル定数 & ベースパラメータ -----------------------
K = M = 4
MU_TILDE = 0.03
ALPHA_GRID = np.arange(0.1,1+0.001,0.001)  # 0〜1 を 0.01 刻み
#   （以下は設計書どおり。ご自身のパラメータ設定に合わせて下さい）
r_base  = np.array([0.02, 0.03, 0.04, 0.05])
sigma_base = np.array([0.20, 0.25, 0.30, 0.35])
Delta_A = np.array([[+1, 1, -1, -1],
                    [+1, 1, -1, -1],
                    [+2, -1, +2, -2],
                    [+2, -2, +2, -2]], float)
Delta_r = Delta_A/1000; Delta_sigma = Delta_A/100
def block_corr(r_in: float, r_out: float, *, debug: bool = True) -> np.ndarray:
    """
    Two-sector block correlation matrix with unit diagonal.

    Parameters
    ----------
    r_in : float
        Within-sector off-diagonal correlation.
    r_out : float
        Between-sector correlation.
    debug : bool, default False
        If True, print summary statistics for the generated matrix.

    Returns
    -------
    C : (K,K) ndarray
        Correlation matrix with diag(C)=1.
    """
    k = 4  # use outer-scope K from self-test or call site
    C = np.full((k, k), r_out, dtype=float)
    np.fill_diagonal(C, 1.0)

    mid = k // 2
    for i in range(mid):
        for j in range(mid):
            if i != j:
                C[i, j] = r_in
    for i in range(mid, k):
        for j in range(mid, k):
            if i != j:
                C[i, j] = r_in

    if debug:
        print(f"[block_corr] K={k}, r_in={r_in}, r_out={r_out}")
        print("  Off-diag min:", C[np.triu_indices(k, 1)].min(),
              "max:", C[np.triu_indices(k, 1)].max())
        print("  Diagonal all ones?:", np.allclose(np.diag(C), 1.0))

    return C

C_base = block_corr(0.75,0.50)
tilde_C = [block_corr(0.85,0.40), block_corr(0.65,0.40),
           block_corr(0.65,0.60), block_corr(0.85,0.60)]
V_base  = np.diag(sigma_base) @ C_base @ np.diag(sigma_base)
df = af.run_comparison_analysis_sweep(
    alpha_grid=ALPHA_GRID,
    r_base=r_base, sigma_base=sigma_base, C_base=C_base, V_base=V_base,
    Delta_r_m=Delta_r, Delta_sigma_m=Delta_sigma, tilde_C_list=tilde_C
)
df.to_csv('result.csv')

[block_corr] K=4, r_in=0.75, r_out=0.5
  Off-diag min: 0.5 max: 0.75
  Diagonal all ones?: True
[block_corr] K=4, r_in=0.85, r_out=0.4
  Off-diag min: 0.4 max: 0.85
  Diagonal all ones?: True
[block_corr] K=4, r_in=0.65, r_out=0.4
  Off-diag min: 0.4 max: 0.65
  Diagonal all ones?: True
[block_corr] K=4, r_in=0.65, r_out=0.6
  Off-diag min: 0.6 max: 0.65
  Diagonal all ones?: True
[block_corr] K=4, r_in=0.85, r_out=0.6
  Off-diag min: 0.6 max: 0.85
  Diagonal all ones?: True


ERROR:analysis_flow:Param‑gen failed (α=1.000, AllVary): alpha must be in [0,1]
Traceback (most recent call last):
  File "/content/analysis_flow.py", line 175, in run_comparison_analysis_sweep
    R_α, Σ_α, V_α, _ = gen_fn(α_val)
                      ^^^^^^^^^^^^^
  File "/content/analysis_flow.py", line 152, in <lambda>
    "AllVary": lambda α: generate_params_all_vary(
                         ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/utilities.py", line 93, in generate_params_all_vary
    C_alpha, lam = mix_corr(C_base, tilde_C_list[m], alpha, beta=beta_corr)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/utilities.py", line 49, in mix_corr
    raise ValueError("alpha must be in [0,1]")
ValueError: alpha must be in [0,1]
ERROR:analysis_flow:Param‑gen failed (α=1.000, CVary): alpha must be in [0,1]
Traceback (most recent call last):
  File "/content/analysis_flow.py", line 175, in run_comparison_analysis_sweep
    R_α, Σ_α, V_α, _ = ge